In [13]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.graph_objects as go
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.sequence import TimeseriesGenerator

In [3]:
filename = "GOOG.csv"
df = pd.read_csv(filename)
look_back = 15
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       253 non-null    object 
 1   Open       253 non-null    float64
 2   High       253 non-null    float64
 3   Low        253 non-null    float64
 4   Close      253 non-null    float64
 5   Adj Close  253 non-null    float64
 6   Volume     253 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 14.0+ KB
None


In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)
df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

In [5]:
# pakt de close data en reshaped het
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

# benoemd de plit procentage
split_percent = 0.80
split = int(split_percent*len(close_data))

# split het in training en test data
close_train = close_data[:split]
close_test = close_data[split:]

# voegt de datum toe aan de gesplite data?
date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

202
51


In [6]:

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [15]:
model = Sequential()
model.add(
    LSTM(64,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_25976\1652816444.py:11: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/25
10/10 [==============================] - 1s 7ms/step - loss: 441462.9375
Epoch 2/25
10/10 [==============================] - 0s 6ms/step - loss: 34502.9336
Epoch 3/25
10/10 [==============================] - 0s 6ms/step - loss: 13292.3076
Epoch 4/25
10/10 [==============================] - 0s 6ms/step - loss: 1555622.1250
Epoch 5/25
10/10 [==============================] - 0s 6ms/step - loss: 2432927.2500
Epoch 6/25
10/10 [==============================] - 0s 7ms/step - loss: 2706991.5000
Epoch 7/25
10/10 [==============================] - 0s 7ms/step - loss: 702315.1250
Epoch 8/25
10/10 [==============================] - 0s 6ms/step - loss: 34136.8516
Epoch 9/25
10/10 [==============================] - 0s 7ms/step - loss: 104640.0000
Epoch 10/25
10/10 [==============================] - 0s 7ms/step - loss: 711310.1250
Epoch 11/25
10/10 [==============================] - 0s 7ms/step - loss: 415342.5312
Epoch 12/25
10/10 [==============================] - 0s 7ms/step - loss: 2

In [24]:
prediction = model.predict(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()